In [ ]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt 
import numpy as np
%matplotlib inline
import torch
import torch.nn as nn

## Warmup exercises

In [ ]:
'''
What the output would be in the case that you apply max pooling with kernel size (i.e., dimension of the filter) 
equals 3 and stride equals 2? You can either calculate it manually by hand or via Pytorch functions.
'''
x=torch.tensor([[[[ 2.6987, 82.5899, 76.2929, 26.5976, 13.1088],
          [23.0795, 97.8725, 76.6898, 88.7104, 51.1192],
          [95.8796, 85.6584, 43.7259,  2.2553, 97.0426],
          [99.6418, 38.8225, 80.7951, 33.3520, 83.8901],
          [20.7309, 81.9728, 61.7567, 87.1467, 10.1192]]]])

##TODO

In [ ]:
""" Given a random input as in the example, find the kernel size and the stride in order to have an output of 
torch.Size([16, 20, 25, 15]). For more details look at the corresponding functions from Pytorch."""

m = nn.MaxPool2d() ##TODO add the appropriate parameters for the MaxPool2d module
input = torch.randn(16, 20, 100, 60)
output = m(input)
output.shape

In [ ]:
""" Given a random input as in the example, find corresponding conv2d parameters to have an ouput 
of torch.Size([20, 10, 10, 20]). For more details look at the corresponding functions from Pytorch."""
m = nn.Conv2d() ##TODO add the appropriate parameters for the Conv2d module
input = torch.randn(20, 16, 50, 100)
output = m(input)
print (output.shape)

## Exercise 1

In [ ]:
# load the digits dataset 
digits = load_digits()
x = digits.data
y = digits.target
n_samples, n_features = x.shape

##TODO reshape your data in such a way that is appropriate as input to the Conv2d module 

print("data shape: ",x.shape)
print("class shape: ",y.shape)

N_train = int(0.8 * x.shape[0])
x_train = x[:N_train,:]
y_train = y[:N_train]
x_test = x[N_train:,:]
y_test = y[N_train:]

In [ ]:
plt.matshow(digits.images[0])
plt.show()

In [ ]:
X_train = torch.tensor(x_train, dtype=torch.float)
Y_train = torch.tensor(y_train)
X_val = torch.tensor(x_test, dtype=torch.float)
Y_val = torch.tensor(y_test)

In [ ]:
# create Dataset object to support batch training
class Dataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features             
        self.labels = labels
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return (self.features[idx], self.labels[idx])

In [ ]:
class Net(nn.Module):
    def __init__(self,n_classes):
        super().__init__()
        '''
        Fill in the Net class in order to build a model that consists of 2 convolutional layers, 
        one max pooling layer, and three linear layers.
        Note that the last linear layer should transform your output to the number of classes. 
        After each linear layer and each convolutional layer use a relu activate function.
        Hint! After the pooling operation you should flatten your data in order to use them as input into the 
        fully connected layer
        
        '''
        ##TODO fill in your code for the CNN architecture
        

    def forward(self, x):

        return x


In [ ]:
def get_predictions(model,data):
    return model(data).numpy().argmax(axis=1)    
def compute_accuracy(predictions,ground_truth):
    # Find the prediction (as the classes with highest probabilities)
    return (predictions == ground_truth.numpy()).mean()

In [ ]:
def run_model(func,optimizer,learning_rate,num_epochs):
    model = func(len(list(set(Y_train.numpy()))))
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(),lr=learning_rate)
    batch_size = 16

    # enable batching of training data
    train_dataset = Dataset(X_train, Y_train)
    dataloader = torch.utils.data.DataLoader(train_dataset,
                                       batch_size=batch_size,
                                       shuffle=True)

    # keep the accuracy values for each training step
    val_accs = np.zeros(num_epochs)

    for i_epoch in range(num_epochs):
        model.train()
        for i_batch, (X_batch, y_batch) in enumerate(dataloader):
            model.zero_grad()  # reset model gradients
            output = model(X_batch)  # conduct forward pass  

            loss=criterion(output, y_batch) 

            loss.backward()  # backpropogate loss to calculate gradients
            optimizer.step()  # update model weights

        with torch.no_grad():  # no need to calculate gradients when assessing accuracy

            model.eval()        
            pred_train = get_predictions(model,X_train)        
            train_acc = compute_accuracy(pred_train, Y_train)
            #print("Training accuracy: {}".format(train_acc))       
            pred_val = get_predictions(model,X_val)        
            val_acc = compute_accuracy(pred_val, Y_val)
            val_accs[i_epoch]=val_acc
            #print("Validation accuracy: {}".format(val_acc))
    
    return val_accs


In [ ]:
optimizer=torch.optim.Adam
val_acc=run_model(Net,optimizer,learning_rate=0.001,num_epochs=50)
plt.plot(val_acc)  
plt.xlabel('Epochs')
plt.ylabel('Val Accuracy')
plt.show()

In [ ]:
"""
TODO:
Plot the validation accuracy using different learning rates of [1e-1, 1e-2, 1e-3, 1e-4, 1e-5] by keeping the rest 
of the hyperparameters fixed. 
Re-use the code from the previous lab.
"""


In [ ]:
"""
TODO:

Plot the validation accuracy using different optimizers (Adam, SGD, Adagrad, Adadelta, RMSprop) by keeping the rest
of the hyperparameters fixed.
Re-use the code from the previous lab.
"""